In [ ]:
$files ??= Get-ChildItem c:\ -depth 2

Get-ChildItem: Access to the path 'C:\Users\postgres' is denied.
Get-ChildItem: Access to the path 'C:\Windows\CSC' is denied.
Get-ChildItem: Access to the path 'C:\Windows\InfusedApps' is denied.
Get-ChildItem: Access to the path 'C:\Windows\LiveKernelReports' is denied.
Get-ChildItem: Access to the path 'C:\Windows\ModemLogs' is denied.
Get-ChildItem: Access to the path 'C:\Windows\Prefetch' is denied.
Get-ChildItem: Access to the path 'C:\Windows\ServiceState' is denied.
Get-ChildItem: Access to the path 'C:\Windows\SystemTemp' is denied.
Get-ChildItem: Access to the path 'C:\Windows\Temp' is denied.
31508


In [ ]:
$Files.count

31508


In [ ]:
$Files.GetType() | ft -auto
$Files.GetType().Name
$Files.GetType().FullName


IsPublic IsSerial Name     BaseType
-------- -------- ----     --------
True     True     Object[] System.Array

Object[]
System.Object[]


In [ ]:
$Files.PsTypeNames -join ', ' 

System.Object[], System.Array, System.Object


`Object[]` means that it is an array of `[Object]` 
We need to index it to test the array's elements type

In [ ]:
$Files[0].GetType().FullName

System.IO.DirectoryInfo


In [ ]:
# Tip, these will never throw an error, no matter the type
@($files)[0].GetType().Name
$x = $null
@($x)[0]


DirectoryInfo


In [ ]:
# but calling a method on null will
@($x)[0].ToString()
@($null)[0].ToString()

InvalidOperation: 
Line |
   2 |  @($x)[0].ToString()
     |  ~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.
InvalidOperation: 
Line |
   3 |  @($null)[0].ToString()
     |  ~~~~~~~~~~~~~~~~~~~~~~
     | You cannot call a method on a null-valued expression.


In [ ]:
$files | Get-Unique -OnType